In [1]:
import sys
import import_ipynb
import numpy as np
from pathlib import Path
from scipy.sparse.linalg import eigsh

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_processing import vectorial_matrices_assembly as assembly
    print("Modules imports were successful!")
except ModuleNotFoundError as en:
    print(f"Modules were not found: {en}")
except ImportError as en:
    print(f"Error in import: {en}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\05_GIT\PPGEE-2024.2-MEF
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


# Project 4: Rectangular Guide

# `setTypeProblem()`

In [2]:
def setTypeProblem(FINITE_ELEMENT, h):
    # Create mesh using gmsh
    BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'Ez_0'}]
    MATERIAL = [{'tag': 201, 'name': 'free_space', 
                'relative_magnetic_permeability': 1, 'relative_electric_permittivity': 1}]
    
    # Create mesh Structure Data from gmsh
    mesh_data = ps.create_rectangular_guide(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=False)

    # Get FEM solution
    Sgr, fgr, Mgr = assembly.reduced_global_matrices(FINITE_ELEMENT, mesh_data)
    
    # Resolver o problema de autovalor generalizado
    eigenvalues, _ = eigsh(Sgr, k=30, M=Mgr, which='SM')

    # Definir um limiar para eliminar erros de truncamento
    positive_indices = np.where(eigenvalues > 1e-8)[0]

    # Filtrar autovalores positivos e ordenar
    sp_eigenvalues = eigenvalues[positive_indices]

    # Ordenar os autovalores positivos e ajustar os autovetores na mesma ordem
    sp_eigenvalues = sp_eigenvalues[np.argsort(sp_eigenvalues)]

    return sp_eigenvalues, mesh_data

# Convergence Analysis

In [3]:
element_types = [('Triangle', 1)]
mesh_sizes = [1E-2 / (2**i) for i in range(0, 3)]

# Inicialização dos dicionários para armazenar erros acumulados para cada tipo de elemento
errors = {et: {} for et in element_types}
num_modes = {et: {} for et in element_types}

# Modos analíticos
te_modes = ps.waveguide_modes(mn_max=5, mode_type='TE')

for h in mesh_sizes:  
    for FINITE_ELEMENT in element_types: 
        print(f"\nCalculating for Mesh size h={h} and element {FINITE_ELEMENT}.")
        # Resolve a solução numérica para o tipo de elemento e refinamento de malha
        eigenvalues, mesh_data = setTypeProblem(FINITE_ELEMENT, h)  

        # Preenche o dicionário dos autovalores
        num_modes[FINITE_ELEMENT][h] = eigenvalues

        # Acumula os erros nos dicionários correspondentes
        errors[FINITE_ELEMENT][h] = ps.calculate_error(te_modes, eigenvalues)


5 menores autovalores analíticos dos modos TE:
TE_10: kc2 = 1542.126 rad2/m2
TE_01: kc2 = 6168.503 rad2/m2
TE_20: kc2 = 6168.503 rad2/m2
TE_11: kc2 = 7710.628 rad2/m2
TE_21: kc2 = 12337.006 rad2/m2

Calculating for Mesh size h=0.01 and element ('Triangle', 1).
Modo TE_10: Erro percentual = 0.0375
Modo TE_01: Erro percentual = 0.2318
Modo TE_20: Erro percentual = 0.1563
Modo TE_11: Erro percentual = 0.1692
Modo TE_21: Erro percentual = 0.1482

Calculating for Mesh size h=0.005 and element ('Triangle', 1).
Modo TE_10: Erro percentual = 0.0034
Modo TE_01: Erro percentual = 0.0139
Modo TE_20: Erro percentual = 0.0290
Modo TE_11: Erro percentual = 0.0029
Modo TE_21: Erro percentual = 0.0168

Calculating for Mesh size h=0.0025 and element ('Triangle', 1).
Modo TE_10: Erro percentual = 0.0005
Modo TE_01: Erro percentual = 0.0008
Modo TE_20: Erro percentual = 0.0031
Modo TE_11: Erro percentual = 0.0005
Modo TE_21: Erro percentual = 0.0010


# Convergence Order, $\alpha$

In [4]:
lambdas = num_modes[('Triangle', 1)]
table = {mode: {} for mode in te_modes.keys()}
for i, mode in enumerate(table.keys()):
    table[mode]['analytic'] = te_modes[mode]    
    table[mode]['lh1'] = lambdas[mesh_sizes[0]][i]   
    table[mode]['lh2'] = lambdas[mesh_sizes[1]][i]   
    table[mode]['lh3'] = lambdas[mesh_sizes[2]][i]   
    table[mode]['alfa'] = np.log(
        np.abs(table[mode]['lh1'] - table[mode]['lh2'])/np.abs(table[mode]['lh2'] - table[mode]['lh3'])) / np.log(2)  
table

{'TE_10': {'analytic': 1542.1256876702123,
  'lh1': 1542.7036316684926,
  'lh2': 1542.1778419176333,
  'lh3': 1542.1339026540281,
  'alfa': 3.5809034235920714},
 'TE_01': {'analytic': 6168.502750680849,
  'lh1': 6154.205470773092,
  'lh2': 6169.363131003612,
  'lh3': 6168.45188592002,
  'alfa': 4.056064137178605},
 'TE_20': {'analytic': 6168.502750680849,
  'lh1': 6178.145340568397,
  'lh2': 6170.292318377429,
  'lh3': 6168.693654115591,
  'alfa': 2.2963809860312687},
 'TE_11': {'analytic': 7710.628438351061,
  'lh1': 7697.584307284608,
  'lh2': 7710.407050883695,
  'lh3': 7710.593271737197,
  'alfa': 6.1055465301008836},
 'TE_21': {'analytic': 12337.0055013617,
  'lh1': 12355.287456957327,
  'lh2': 12334.936524850613,
  'lh3': 12337.130060509464,
  'alfa': 3.2137648092549895}}

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.